In [173]:
import pymysql
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
from datetime import datetime
import time
from ggplot import *

In [102]:
conn = pymysql.connect('rdsl1bdoa7h7d5ko2hkno.mysql.rds.aliyuncs.com',
                       user='bossaccess',
                       passwd = 'master-502',
                       db = 'chekumaster',
                       charset='utf8')
sql = "select * from cheku_spec_order where create_time > '2017-01-01'"

In [103]:
df = pd.read_sql(sql,conn)

In [104]:
data = df[['create_time','order_num','order_info','buy_num',
           'cheku_price','username','earnest',
           'order_status','car_id','bag_id','create_id','out_color']]

In [105]:
sql = '''SELECT cheku_car_model.id, cheku_car_brand.name, cheku_car_series.name, cheku_car_model.name, cheku_car_msrp.price
FROM cheku_car_msrp INNER JOIN (cheku_car_model INNER JOIN 
(cheku_car_brand INNER JOIN cheku_car_series ON cheku_car_brand.id = cheku_car_series.brand_id) 
ON cheku_car_model.series_id = cheku_car_series.id) ON cheku_car_msrp.model_id = cheku_car_model.id;
'''

In [106]:
carinfo = pd.read_sql(sql,conn)

In [107]:
sql = '''SELECT cheku_spec_sale_req.id, cheku_spec_sale_req.msrp
FROM cheku_spec_sale_req;
'''

In [108]:
bagidf3 = pd.read_sql(sql,conn)

In [109]:
data2 = pd.merge(data,bagidf3,left_on = 'car_id',right_on = 'id' , how = 'left') 

In [110]:
cols = 'create_time', 'order_num', 'order_info','msrp', 'buy_num', 'cheku_price','username', 'earnest', 'order_status', 'car_id', 'bag_id', 'id'
#changel 'msrp' col
data2_msrp = data2['msrp']
data2 = data2.drop('msrp',axis =1)
data2.insert(3,'msrp',data2_msrp)
data2 = data2.drop('bag_id',axis = 1)

In [111]:
temp = pd.DataFrame((x.split(':') for x in data2.order_info),columns=['brand','m','series','model'])
temp = temp.drop('m',axis = 1)
data2.insert(2,'brand',temp['brand'])
data2.insert(3,'series',temp['series'])
data2.insert(4,'model',temp['model'])
data2 = data2.drop('order_info',axis = 1)

In [112]:
data2['msrp'] = [int(x) / 100 for x in data2['msrp']]

In [113]:
status = Series(range(-3,10))
status_info = Series(['融资审核','','用户取消','待付定金','已付定金','已分配VIN号',
                     '定金凭证已上传','已付首付','全款凭证已上传','首付凭证已上传',
                     '已完成','已付全款','融资审核中'])
status = DataFrame({'status':status,'status_info':status_info})
data2 = pd.merge(data2,status,left_on = 'order_status',right_on = 'status',how = 'left')

In [114]:
sql = ''

In [115]:
data2

create_time      order_num brand series                   model  \
0  2017-01-20 11:26:54  T170120288465   雪佛兰    科鲁兹     2015款 1.5L 经典 SE AT   
1  2017-02-08 15:35:02  T170208504028   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   
2  2017-02-08 15:55:44  T170208768356   雪佛兰     赛欧    2015款 赛欧3 1.5L 手动理想版   
3  2017-02-08 16:09:10  T170208286047   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   
4  2017-02-08 16:11:34  T170208652519   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   
5  2017-02-08 16:15:39  T170208744523   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   
6  2017-02-09 14:37:43  T170209532288   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   
7  2017-02-09 16:20:03  T170209342311   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   
8  2017-02-09 17:09:08  T170209263530   雪佛兰    科沃兹        2016款 1.5L 自动欣尚版   
9  2017-02-15 09:27:51  T170215912439   雪佛兰    科沃兹        2016款 1.5L 手动欣悦版   
10 2017-02-23 10:16:41  T170223419085   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座   
11 2017-02-23 10:22:42  T170223268817   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座   
12 2017-02-23 10:31:01  T170223729133   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座   
13 2017-02-23 10:32:21  T170223401133   雪佛兰    科帕奇     2017款 2.4L 两驱豪华版 7座   
14 2017-02-23 13:28:09  T170223485974   雪佛兰  迈锐宝XL        2016款 2.5L 自动锐尊版   
15 2017-02-27 15:32:17  T170227303925    别克     英朗         2017款 15N 自动进取型   

    msrp  buy_num cheku_price username  earnest  order_status  car_id  \
0   1099        1       86900      宋广源     2000             1     699   
1    639        1       50900      王阿蒙        0             1     700   
2    679        1       53900      王阿蒙        0             1     701   
3    889        1       79900      王阿蒙        0             1     702   
4    889        2      159800      王阿蒙        0             1     703   
5    999        3      269700      王阿蒙        0             1     704   
6    639        6      303600      张云范    12000            -2     705   
7    999       15     1348500      宋广源    75000            -2     706   
8   1099        5      494500      宋广源    25000            -2     707   
9    909        1       81900      王阿蒙        0             1     708   
10  2099        1      183900       何进     5000             0     709   
11  2099        1      183900      杨小姐     3000             0     710   
12  2099        2      367800     北美强生        0             1     711   
13  1899        1      164900     北美强生        0             1     712   
14  2499        1      213900      王阿蒙        0             1     713   
15  1199        1       92900      王阿蒙     2000             1     714   

    create_id out_color   id  status status_info  
0          79       珍珠白  699       1        已付定金  
1          79       茉莉白  700       1        已付定金  
2          79       茉莉白  701       1        已付定金  
3          79       玛瑙红  702       1        已付定金  
4          79       珍珠白  703       1        已付定金  
5          79       珍珠白  704       1        已付定金  
6          79       茉莉白  705      -2              
7          79       珍珠白  706      -2              
8          79       珍珠白  707      -2              
9          79       珍珠白  708       1        已付定金  
10         20       旷夜黑  709       0        待付定金  
11         20       摩卡棕  710       0        待付定金  
12         20       摩卡棕  711       1        已付定金  
13         20       摩卡棕  712       1        已付定金  
14         79       州际灰  713       1        已付定金  
15         79       雪域白  714       1        已付定金

In [116]:
data2 = data2.drop(['order_status','status','car_id','id'],axis = 1)

In [117]:
sql = 'SELECT id,chnname FROM chekumaster.cheku_user;'
user = pd.read_sql(sql,conn)


In [118]:
data2 = pd.merge(data2,user,left_on= 'create_id',right_on = 'id',how='inner')
data2 = data2.drop('create_id',axis = 1)

In [119]:
times = pd.to_datetime(data2.create_time)
data2['create_time'] = [time.strftime('%Y-%m-%d') for time in times]

In [120]:
data2

create_time      order_num brand series                   model  msrp  \
0   2017-01-20  T170120288465   雪佛兰    科鲁兹     2015款 1.5L 经典 SE AT  1099   
1   2017-02-08  T170208504028   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639   
2   2017-02-08  T170208768356   雪佛兰     赛欧    2015款 赛欧3 1.5L 手动理想版   679   
3   2017-02-08  T170208286047   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889   
4   2017-02-08  T170208652519   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889   
5   2017-02-08  T170208744523   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999   
6   2017-02-09  T170209532288   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639   
7   2017-02-09  T170209342311   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999   
8   2017-02-09  T170209263530   雪佛兰    科沃兹        2016款 1.5L 自动欣尚版  1099   
9   2017-02-15  T170215912439   雪佛兰    科沃兹        2016款 1.5L 手动欣悦版   909   
10  2017-02-23  T170223485974   雪佛兰  迈锐宝XL        2016款 2.5L 自动锐尊版  2499   
11  2017-02-27  T170227303925    别克     英朗         2017款 15N 自动进取型  1199   
12  2017-02-23  T170223419085   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099   
13  2017-02-23  T170223268817   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099   
14  2017-02-23  T170223729133   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099   
15  2017-02-23  T170223401133   雪佛兰    科帕奇     2017款 2.4L 两驱豪华版 7座  1899   

    buy_num cheku_price username  earnest out_color status_info  id chnname  
0         1       86900      宋广源     2000       珍珠白        已付定金  79     赵有民  
1         1       50900      王阿蒙        0       茉莉白        已付定金  79     赵有民  
2         1       53900      王阿蒙        0       茉莉白        已付定金  79     赵有民  
3         1       79900      王阿蒙        0       玛瑙红        已付定金  79     赵有民  
4         2      159800      王阿蒙        0       珍珠白        已付定金  79     赵有民  
5         3      269700      王阿蒙        0       珍珠白        已付定金  79     赵有民  
6         6      303600      张云范    12000       茉莉白              79     赵有民  
7        15     1348500      宋广源    75000       珍珠白              79     赵有民  
8         5      494500      宋广源    25000       珍珠白              79     赵有民  
9         1       81900      王阿蒙        0       珍珠白        已付定金  79     赵有民  
10        1      213900      王阿蒙        0       州际灰        已付定金  79     赵有民  
11        1       92900      王阿蒙     2000       雪域白        已付定金  79     赵有民  
12        1      183900       何进     5000       旷夜黑        待付定金  20      陈汀  
13        1      183900      杨小姐     3000       摩卡棕        待付定金  20      陈汀  
14        2      367800     北美强生        0       摩卡棕        已付定金  20      陈汀  
15        1      164900     北美强生        0       摩卡棕        已付定金  20      陈汀

In [121]:
data2 = data2.drop(['order_num','id'],axis = 1)

In [122]:
data2

create_time brand series                   model  msrp  buy_num  \
0   2017-01-20   雪佛兰    科鲁兹     2015款 1.5L 经典 SE AT  1099        1   
1   2017-02-08   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639        1   
2   2017-02-08   雪佛兰     赛欧    2015款 赛欧3 1.5L 手动理想版   679        1   
3   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889        1   
4   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889        2   
5   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999        3   
6   2017-02-09   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639        6   
7   2017-02-09   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999       15   
8   2017-02-09   雪佛兰    科沃兹        2016款 1.5L 自动欣尚版  1099        5   
9   2017-02-15   雪佛兰    科沃兹        2016款 1.5L 手动欣悦版   909        1   
10  2017-02-23   雪佛兰  迈锐宝XL        2016款 2.5L 自动锐尊版  2499        1   
11  2017-02-27    别克     英朗         2017款 15N 自动进取型  1199        1   
12  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        1   
13  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        1   
14  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        2   
15  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 两驱豪华版 7座  1899        1   

   cheku_price username  earnest out_color status_info chnname  
0        86900      宋广源     2000       珍珠白        已付定金     赵有民  
1        50900      王阿蒙        0       茉莉白        已付定金     赵有民  
2        53900      王阿蒙        0       茉莉白        已付定金     赵有民  
3        79900      王阿蒙        0       玛瑙红        已付定金     赵有民  
4       159800      王阿蒙        0       珍珠白        已付定金     赵有民  
5       269700      王阿蒙        0       珍珠白        已付定金     赵有民  
6       303600      张云范    12000       茉莉白                 赵有民  
7      1348500      宋广源    75000       珍珠白                 赵有民  
8       494500      宋广源    25000       珍珠白                 赵有民  
9        81900      王阿蒙        0       珍珠白        已付定金     赵有民  
10      213900      王阿蒙        0       州际灰        已付定金     赵有民  
11       92900      王阿蒙     2000       雪域白        已付定金     赵有民  
12      183900       何进     5000       旷夜黑        待付定金      陈汀  
13      183900      杨小姐     3000       摩卡棕        待付定金      陈汀  
14      367800     北美强生        0       摩卡棕        已付定金      陈汀  
15      164900     北美强生        0       摩卡棕        已付定金      陈汀

In [123]:
data2['cheku_price'] = [int(x) for x in data2['cheku_price']]


In [124]:
data2

create_time brand series                   model  msrp  buy_num  \
0   2017-01-20   雪佛兰    科鲁兹     2015款 1.5L 经典 SE AT  1099        1   
1   2017-02-08   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639        1   
2   2017-02-08   雪佛兰     赛欧    2015款 赛欧3 1.5L 手动理想版   679        1   
3   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889        1   
4   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣享版   889        2   
5   2017-02-08   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999        3   
6   2017-02-09   雪佛兰     赛欧  2016款 赛欧3 1.3L 手动舒适天窗版   639        6   
7   2017-02-09   雪佛兰    科沃兹        2016款 1.5L 自动欣悦版   999       15   
8   2017-02-09   雪佛兰    科沃兹        2016款 1.5L 自动欣尚版  1099        5   
9   2017-02-15   雪佛兰    科沃兹        2016款 1.5L 手动欣悦版   909        1   
10  2017-02-23   雪佛兰  迈锐宝XL        2016款 2.5L 自动锐尊版  2499        1   
11  2017-02-27    别克     英朗         2017款 15N 自动进取型  1199        1   
12  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        1   
13  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        1   
14  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 四驱旗舰版 7座  2099        2   
15  2017-02-23   雪佛兰    科帕奇     2017款 2.4L 两驱豪华版 7座  1899        1   

    cheku_price username  earnest out_color status_info chnname  
0         86900      宋广源     2000       珍珠白        已付定金     赵有民  
1         50900      王阿蒙        0       茉莉白        已付定金     赵有民  
2         53900      王阿蒙        0       茉莉白        已付定金     赵有民  
3         79900      王阿蒙        0       玛瑙红        已付定金     赵有民  
4        159800      王阿蒙        0       珍珠白        已付定金     赵有民  
5        269700      王阿蒙        0       珍珠白        已付定金     赵有民  
6        303600      张云范    12000       茉莉白                 赵有民  
7       1348500      宋广源    75000       珍珠白                 赵有民  
8        494500      宋广源    25000       珍珠白                 赵有民  
9         81900      王阿蒙        0       珍珠白        已付定金     赵有民  
10       213900      王阿蒙        0       州际灰        已付定金     赵有民  
11        92900      王阿蒙     2000       雪域白        已付定金     赵有民  
12       183900       何进     5000       旷夜黑        待付定金      陈汀  
13       183900      杨小姐     3000       摩卡棕        待付定金      陈汀  
14       367800     北美强生        0       摩卡棕        已付定金      陈汀  
15       164900     北美强生        0       摩卡棕        已付定金      陈汀

In [125]:
grouped = data2[['buy_num','cheku_price']].groupby(data2['chnname'])

In [126]:
grouped.sum()

buy_num  cheku_price
chnname                      
赵有民           38      3236400
陈汀             5       900500

In [128]:
data2.index = data2.create_time

In [130]:
data2.index = pd.to_datetime(data2.index)

In [187]:
temp = data2[['buy_num','cheku_price']].groupby([data2.series,data2.msrp]).resample('MS').sum().unstack()

In [191]:
temp.fillna('')

buy_num            cheku_price            
create_time 2017-01-01 2017-02-01  2017-01-01  2017-02-01
series msrp                                              
科帕奇    1899                     1                  164900
       2099                     4                  735600
科沃兹    889                      3                  239700
       909                      1                   81900
       999                     18              1.6182e+06
       1099                     5                  494500
科鲁兹    1099          1                  86900            
英朗     1199                     1                   92900
赛欧     639                      7                  354500
       679                      1                   53900
迈锐宝XL  2499                     1                  213900

In [179]:
ggplot(temp,aes(temp.index[2]))

Exception: Invalid column: '(u'\u79d1\u6c83\u5179', 889, Timestamp('2017-02-01 00:00:00'))'
did you mean one of the following:


In [186]:
temp.index[1:]

MultiIndex(levels=[[u'科帕奇', u'科沃兹', u'科鲁兹', u'英朗', u'赛欧', u'迈锐宝XL'], [639, 679, 889, 909, 999, 1099, 1199, 1899, 2099, 2499], [2017-01-01 00:00:00, 2017-02-01 00:00:00]],
           labels=[[0, 1, 1, 1, 1, 2, 3, 4, 4, 5], [8, 2, 3, 4, 5, 5, 6, 0, 1, 9], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]],
           names=[u'series', u'msrp', u'create_time'])